# Tâche 5

In [90]:
#import 
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from jupyter_jsmol.pymatgen import quick_view,JsmolView
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import Image
from ipywidgets import Layout, widgets, interact
import matplotlib.pyplot as plt





In [91]:
mp_key = "0xG7dCsc4AADF0PgqeiDeWIJa0gtg1Pd"
mp_id = "mp-9166"

In [92]:
with MPRester(mp_key) as m:
    struct = m.get_structure_by_material_id(mp_id)
    
conv_struct = SpacegroupAnalyzer(struct).get_conventional_standard_structure()

symmops = SpacegroupAnalyzer(conv_struct).get_space_group_operations()


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Data 


In [93]:
#indice des atomes du réseau : 

Mg_atom = []
C_atom = []
N_atom = []

for i  in range (len(conv_struct.sites)): 
    if (conv_struct.sites[i].label == 'Mg'): 
        Mg_atom.append(i)
    elif (conv_struct.sites[i].label == 'N'): 
        N_atom.append(i)
    else : 
        C_atom.append(i)


# 1)  Première opération

Nous avons choisi pour cette première opération de manière aléatoire un atome de magnésium du réseau. On observe que pour cette première opération, notre vecteur de translation $\tau$ nul. Cela signifie qu'il n'y a pas de translation associée à l'opération. L'opération est dès lors purement rotationnelle. Par ailleurs, en calculant le detérminant de la matrice rot, nous avons que celui-ci vaut 1, ce qui implique que l'opération conserve la chiralité de la molécule. L'axe de rotation associé à cette opération de symétrie est l'axe z : \begin{bmatrix} 0 & 0 & 1 \end{bmatrix} 

Enfin, en observant la matrice rot, on remarque que l'opération de symétrie inverse les coordonnées selon Z et échange les coordonnées selon X et Y.

 

In [94]:
i_symmop = 7
#indice random d'un atome de Magnesium du réseau : 
i_atom = np.random.choice(Mg_atom)+1
symmop = symmops[i_symmop-1] 
print(f"Opérateur de symétrie :\n {symmop}\n")

Opérateur de symétrie :
 Rot:
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]



In [95]:
pos_init = conv_struct.sites[i_atom-1].frac_coords
pos_final = symmop.operate(pos_init)
##affichage :  
print("1er opération de symétrie (inversion): \n")
print(f" Position de l'atome de départ de {conv_struct.sites[i_atom-1].label} :\n  {pos_init}\n Position de l'atome de {conv_struct.sites[i_atom-1].label} après opération de symétrie:\n {pos_final} ")

1er opération de symétrie (inversion): 

 Position de l'atome de départ de Mg :
  [0.33333333 0.66666667 0.66666667]
 Position de l'atome de Mg après opération de symétrie:
 [ 0.66666667  0.33333333 -0.66666667] 


In [98]:
view = quick_view(conv_struct,"packed",conventional = True)

display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [99]:
view.script('select Magnesium; color green')
view.script('select Nitrogen; color gray')
view.script('select Carbon; color blue')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')
view.script(f"select atomno={i_atom}; color red")

# Changer le fond en blanc
view.script("background white")



# 2) Deuxième opération


Nous avons décidé d'appliquer ce deuxième opérateur de symétrie sur un atome choisi au hasard de Carbone. Le vecteur de translation $\tau$ est toujours nul, ce qui signifie qu'il n'y a toujours pas de translation associé à l'opération. Le déterminant de la matrice rot est quant à lui égale à -1. Cela signifie qu'il s'agit d'un opérateur d'inversion. C'est une opération de deuxième espèce et celle-ci inverse la chiralité de la molécule. En analysant la matrice rot, on observe que chaque coordonnée est inversé par rapport à l'orgine.

In [100]:
#indice random d'un atome de Carbone du réseau : 
i_atom = np.random.choice(C_atom)+1
i_symmop = 2
symmop = symmops[i_symmop-1] 
print(f"Opérateur de symétrie :\n {symmop}\n")

Opérateur de symétrie :
 Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]



In [101]:
pos_init = conv_struct.sites[i_atom-1].frac_coords
pos_final = symmop.operate(pos_init)
##affichage :  
print("1er opération de symétrie (inversion): \n")
print(f" Position de l'atome de départ de {conv_struct.sites[i_atom-1].label} :\n  {pos_init}\n Position de l'atome de {conv_struct.sites[i_atom-1].label} après opération de symétrie:\n {pos_final} ")

1er opération de symétrie (inversion): 

 Position de l'atome de départ de C :
  [0.33333333 0.66666667 0.16666667]
 Position de l'atome de C après opération de symétrie:
 [-0.33333333 -0.66666667 -0.16666667] 


In [102]:
view1 = quick_view(conv_struct,"packed",conventional = True)

display(view1)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [103]:
view1.script('select Magnesium; color green')
view1.script('select Nitrogen; color gray')
view1.script('select Carbon; color blue')
view1.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')
view1.script(f"select atomno={i_atom}; color red")

view1.script("background white")

# 3) Troisième opération

Enfin, concernant le troisème opérateur, on l'a appliqué à un atome d'azote du réseau. On observe toujours qu'il n'y a pas de translation associé à l'opération. Le déterminant associé à la matrice rot est égale 1 donc l'opérateur conserve la chiralité. 

In [104]:
i_symmop = 5
#indice random d'un atome d'azote du réseau : 
i_atom = np.random.choice(N_atom)+1
symmop = symmops[i_symmop-1] 
print(f"Opérateur de symétrie :\n {symmop}\n")

Opérateur de symétrie :
 Rot:
[[-1.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]



In [105]:
pos_init = conv_struct.sites[i_atom-1].frac_coords
pos_final = symmop.operate(pos_init)
##affichage :  
print("1er opération de symétrie (inversion): \n")
print(f" Position de l'atome de départ de {conv_struct.sites[i_atom-1].label} :\n  {pos_init}\n Position de l'atome de {conv_struct.sites[i_atom-1].label} après opération de symétrie:\n {pos_final} ")

1er opération de symétrie (inversion): 

 Position de l'atome de départ de N :
  [0.66666667 0.33333333 0.91993065]
 Position de l'atome de N après opération de symétrie:
 [-0.33333333 -0.66666667  0.91993065] 


In [106]:
view2 = quick_view(conv_struct,"packed",conventional = True)

display(view2)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [107]:
view2.script('select Magnesium; color green')
view2.script('select Nitrogen; color gray')
view2.script('select Carbon; color blue')
view2.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')
view2.script(f"select atomno={i_atom}; color red")

view2.script("background white")